<a href="https://colab.research.google.com/github/derekphilipau/machinelearningforartists/blob/main/stylegan2_ada_pytorch_all_new2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP (MUST RUN EACH TIME)

## Verify Runtime is GPU

In the menu, select Runtime -> Change Runtime Type and verify you are using the **GPU**.  Also select **High-RAM** if you are using Colab Pro.

The `nvidia-smi` command below should **NOT** display *"NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver."*

In [ ]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-ac66fe9e-512d-a4c1-8a7a-98b1a04308d9)


## Mount your Google Drive

You will be storing the training models and progress images on your Google Drive.  This is very convenient for viewing progress, and if your Colab notebook is disconnected you will not lose your models.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Stylegan2-ada-pytorch Prerequisites

In [ ]:
import torch

Verify the next command results in "1".  If not, go back to the beginning and verify you have a GPU runtime.

In [ ]:
torch.cuda.device_count()

1

In [ ]:
import torchvision

In [ ]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

     |████████████████████████████████| 204kB 13.0MB/s 
     |████████████████████████████████| 112kB 19.9MB/s 
     |████████████████████████████████| 26.9MB 1.4MB/s 


In [ ]:
!apt install imagemagick

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libcupsfilters1 libcupsimage2
  libdjvulibre-text libdjvulibre21 libgs9 libgs9-common libijs-0.35
  libjbig2dec0 liblqr-1-0 libmagickcore-6.q16-3 libmagickcore-6.q16-3-extra
  libmagickwand-6.q16-3 libnetpbm10 libwmf0.2-7 netpbm poppler-data
Suggested packages:
  fonts-noto ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng
  enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance
  sane-utils texlive-base-bin transfig ufraw-batch inkscape libjxr-tools
  libwmf0.2-7-gtk poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono g

## Get the StyleGAN code

In [ ]:
!git clone https://github.com/derekphilipau/stylegan2-ada-pytorch.git

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 128 (delta 27), reused 25 (delta 25), pack-reused 85
Receiving objects: 100% (128/128), 1.12 MiB | 24.93 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [ ]:
mkdir /content/stylegan2-ada-pytorch/datasets

In [ ]:
%cd /content/stylegan2-ada-pytorch

/content/stylegan2-ada-pytorch


# FIRST RUN (RUN ONLY ONCE)

## Create Folders on Your Google Drive

If we accidentally close our browser or the Colab runtime disconnects, we will lose all of our training models and progress images.  Therefore we want to store the training data on our Google Drive.  The following cells will create a new folder on your Google Drive, **MachineLearningForArtists**.  Your training data will be stored in **MachineLearningForArtists/MyProject**

In [ ]:
mkdir /content/drive/MyDrive/MachineLearningForArtists

In [ ]:
mkdir /content/drive/MyDrive/MachineLearningForArtists/MyProject

## Prepare Your Image Dataset for StyleGAN


### Unzip your dataset

***If your images are not zipped, skip this step.***

If your images are in a .zip file, you will need to unzip them.   Copy the path to your .zip file and paste here under `zip_path`. 


In [ ]:
zip_path = '' #@param {type:"string"}

In [ ]:
!unzip "$zip_path"

### Prepare your dataset for StyleGAN


Copy the path to your unzipped dataset folder of images and paste into `dataset_path`

In [ ]:
dataset_path = '/content/drive/MyDrive/iseman data set' #@param {type:"string"}

The following commands will prepare your dataset.  It will crop all of your images to 512x512 pixels.  ***It will NOT overwrite your original images.***

In [ ]:
!mkdir /content/stylegan2-ada-pytorch/datasets/raw

In [ ]:
!cp "$dataset_path"/* /content/stylegan2-ada-pytorch/datasets/raw/

^C


In [ ]:
%cd /content/stylegan2-ada-pytorch/datasets/raw/

/content/stylegan2-ada-pytorch/datasets/raw


In [ ]:
!mogrify -type TrueColor -set colorspace sRGB -colorspace sRGB -resize 512x512 -background white -gravity center -extent 512x512 -format jpg *

In [ ]:
cd /content/stylegan2-ada-pytorch/

/content/stylegan2-ada-pytorch


In [ ]:
!python dataset_tool.py --source=/content/stylegan2-ada-pytorch/datasets/raw/ --dest=./datasets/stylegan_dataset.zip

100% 541/541 [00:10<00:00, 51.52it/s]


In [ ]:
!cp ./datasets/stylegan_dataset.zip /content/drive/MyDrive/MachineLearningForArtists/MyProject/

If you get an error "cannot create directory", it's probably because the folder already exists on your Google Drive and you can ignore the error.

## Train from Scratch

This cell block will train StyleGAN from scratch.  Training from scratch is much slower than using *transfer learning* on a previously trained model.  However, the purpose of this tutorial is to a) try training for the first time and b) notice how the progress images develop over time.

In [ ]:
!python train.py --outdir=/content/drive/MyDrive/MachineLearningForArtists/MyProject --data=./datasets/stylegan_dataset.zip --gpus=1 --cfg=paper512 --mirror=1 --snap=10 --metrics=none


## Go to bed :)

Normal training of a dataset can take days.

Results are stored in your Google Drive under drive/MyDrive/MachineLearningForArtists/MyProject

Each training run is stored in a separate directory.  The initial training run is stored in `00000-stylegan_dataset...`.  If you run training twice, the second run will be stored in `00001-stylegan_dataset...`.  And so on.

Inside the training run directory you will see various files.  `reals.png` shows a sample of the training dataset.  In it you should see various images from your original dataset. `fakes000000.png` is a sample of generated images from the initial model.  `network-snapshot-XXXXXX.pkl` is the actual model which can be used later to generate "fake" images, videos, etc.

As training progresses you will see more `fakes` & `network-snapshot` files created.


# RESUME TRAINING

***Make sure you run the above section "SETUP (MUST RUN EACH TIME)" before resuming training.***


In [ ]:
%cd /content/stylegan2-ada-pytorch

/content/stylegan2-ada-pytorch


Copy previously saved datset

In [ ]:
!cp /content/drive/MyDrive/MachineLearningForArtists/MyProject/stylegan_dataset.zip ./datasets/

If you had to stop training, the browser window closed, or the Colab session timed out (12-24 hours), you can resume training by using a .pkl file stored on your Google Drive.  For your project, this file will be in `drive/MyDrive/MachineLearningForArtists/MyProject/0000X-stylegan_datset...`

In [ ]:
#@title PKL File Path 
#@markdown Copy & Paste .pkl file path here:
pkl_file = '' #@param {type:"string"}


Run the training, resuming with the last save .pkl file.  Results will be stored in your Google Drive under `MachineLearningForArtists/MyProject`

In [ ]:
cd /content/stylegan2-ada-pytorch/

In [ ]:
!python train.py --resume=$pkl_file --outdir=/content/drive/MyDrive/MachineLearningForArtists/MyProject --data=./datasets/stylegan_dataset.zip --gpus=1 --cfg=paper512 --mirror=1 --snap=10 --metrics=none


# OPTIONAL: TRANSFER LEARNING

## OR Use Transfer Learning (Optional)

We don't have to train from scratch.  We can train using *transfer learning*.  We will use a StyleGAN2 model already pre-trained on faces to train a new model that is trained against our image dataset.

### Get the faces model

In [ ]:
%cd /content/stylegan2-ada-pytorch

In [ ]:
!apt-get install megatools

In [ ]:
!megadl https://mega.nz/#!eQdHkShY!8wyNKs343L7YUjwXlEg3cWjqK2g2EAIdYz5xbkPy3ng

### Prepare old Faces model for StyleGAN2-ada-pytorch


In [ ]:
!python legacy.py \
    --source=/content/stylegan2-ada-pytorch/ffhq-512-avg-tpurun1.pkl \
    --dest=stylegan2-ada-pytorch-ffhq-512.pkl

### Train using the faces model with your dataset 

In [ ]:
!python train.py --resume=/content/stylegan2-ada-pytorch/stylegan2-ada-pytorch-ffhq-512.pkl --outdir=/content/drive/MyDrive/MachineLearningForArtists/MyProject --data=./datasets/stylegan_dataset.zip --gpus=1 --cfg=paper512 --mirror=1 --snap=10 --metrics=none
